# 开始采集
之所以叫网络爬虫（Web crawler）是因为它们可以沿着网络爬行。它们的本质就是一种递
归方式。为了找到URL 链接，它们必须首先获取网页内容，检查这个页面的内容，再寻
找另一个URL，然后获取URL 对应的网页内容，不断循环这一过程。

### 3.1　遍历单个域名

即使你没听说过“维基百科六度分隔理论”，也很可能听过“凯文 · 贝肯（Kevin Bacon）
的六度分隔值游戏”。在这两个游戏中，都是把两个不相干的主题（维基百科里是用词条
之间的连接，凯文 · 贝肯的六度分隔值游戏是用出现在同一部电影中的演员来连接）用一
个总数不超过六条的主题连接起来（包括原来的两个主题）。

### 创建一个项目实现“维基百科六度分隔理论”的查找方法.

我们要实现从[埃里克 · 艾德尔的词条页面(Eric_Idle)](https://en.wikipedia.org/wiki/Eric_Idle)开始，经过最少的链接点击次数找到[凯文 · 贝肯(Kevin_Bacon)的词条页面](https://en.wikipedia.org/wiki/Kevin_Bacon)。

你应该已经知道如何写一段获取维基百科网站的任何页面并提取页面链接的Python 代码了：

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup 

html = urlopen('http://en.wikipedia.org/wiki/Kevin_Bacon')
bs = BeautifulSoup(html, 'html.parser')
for link in bs.find_all('a'):
    if 'href' in link.attrs:
        print(link.attrs['href'])

/wiki/Wikipedia:Protection_policy#semi
#mw-head
#p-search
/wiki/Kevin_Bacon_(disambiguation)
/wiki/File:Kevin_Bacon_SDCC_2014.jpg
/wiki/San_Diego_Comic-Con
/wiki/Philadelphia
/wiki/Pennsylvania
/wiki/Kyra_Sedgwick
/wiki/Sosie_Bacon
/wiki/Edmund_Bacon_(architect)
/wiki/Michael_Bacon_(musician)
http://baconbros.com/
#cite_note-1
#cite_note-actor-2
/wiki/Footloose_(1984_film)
/wiki/JFK_(film)
/wiki/A_Few_Good_Men
/wiki/Apollo_13_(film)
/wiki/Mystic_River_(film)
/wiki/Sleepers
/wiki/The_Woodsman_(2004_film)
/wiki/Fox_Broadcasting_Company
/wiki/The_Following
/wiki/HBO
/wiki/Taking_Chance
/wiki/Golden_Globe_Award
/wiki/Screen_Actors_Guild_Award
/wiki/Primetime_Emmy_Award
/wiki/The_Guardian
/wiki/Academy_Award
#cite_note-3
/wiki/Hollywood_Walk_of_Fame
#cite_note-4
/wiki/Social_networks
/wiki/Six_Degrees_of_Kevin_Bacon
/wiki/SixDegrees.org
#cite_note-walk-5
#Early_life_and_education
#Acting_career
#Early_work
#1980s
#1990s
#2000s
#2010s
#Advertising_work
#Personal_life
#Six_Degrees_of_Kevin_Ba

如果你观察生成的一列链接，就会看到你想要的所有词条链接都在里面：“Apollo 13”
“Philadelphia”和“Primetime Emmy Award”，等等。但是，也有一些我们不需要的链接：
```HTML
//wikimediafoundation.org/wiki/Privacy_policy
//en.wikipedia.org/wiki/Wikipedia:Contact_us
```

其实维基百科的每个页面都充满了侧边栏、页眉、页脚链接，以及连接到分类页面、对话
页面和其他不包含词条的页面的链接：
```HTML
/wiki/Category:Articles_with_unsourced_statements_from_April_2014
/wiki/Talk:Kevin_Bacon
```
如果你仔细观察那些指向词条页面（不是指向其他内容页面）的链接，会发现它们都有三个共同点：

- 它们都在id 是bodyContent 的div 标签里
- URL 链接不包含分号
- URL 链接都以/wiki/ 开头


我们可以利用这些规则稍微调整一下代码来获取词条链接：

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

html = urlopen('http://en.wikipedia.org/wiki/Kevin_Bacon')
bs = BeautifulSoup(html, 'html.parser')
for link in bs.find('div', {'id': 'bodyContent'}).find_all(
     'a', href=re.compile('^(/wiki/)((?!:).)*$')):
    if 'href' in link.attrs:
        print(link.attrs['href'])

/wiki/Kevin_Bacon_(disambiguation)
/wiki/San_Diego_Comic-Con
/wiki/Philadelphia
/wiki/Pennsylvania
/wiki/Kyra_Sedgwick
/wiki/Sosie_Bacon
/wiki/Edmund_Bacon_(architect)
/wiki/Michael_Bacon_(musician)
/wiki/Footloose_(1984_film)
/wiki/JFK_(film)
/wiki/A_Few_Good_Men
/wiki/Apollo_13_(film)
/wiki/Mystic_River_(film)
/wiki/Sleepers
/wiki/The_Woodsman_(2004_film)
/wiki/Fox_Broadcasting_Company
/wiki/The_Following
/wiki/HBO
/wiki/Taking_Chance
/wiki/Golden_Globe_Award
/wiki/Screen_Actors_Guild_Award
/wiki/Primetime_Emmy_Award
/wiki/The_Guardian
/wiki/Academy_Award
/wiki/Hollywood_Walk_of_Fame
/wiki/Social_networks
/wiki/Six_Degrees_of_Kevin_Bacon
/wiki/SixDegrees.org
/wiki/Philadelphia
/wiki/Edmund_Bacon_(architect)
/wiki/Pennsylvania_Governor%27s_School_for_the_Arts
/wiki/Bucknell_University
/wiki/Glory_Van_Scott
/wiki/Circle_in_the_Square
/wiki/Nancy_Mills
/wiki/Cosmopolitan_(magazine)
/wiki/Fraternities_and_sororities
/wiki/Animal_House
/wiki/Search_for_Tomorrow
/wiki/Guiding_Light
/wiki/F

如果你运行代码，就会看到维基百科上凯文 ·贝肯词条里所有指向其他词条的链接。

当然，写程序来找出这个静态的维基百科词条里所有的词条链接很有趣，不过没什么实际
用处。我们需要让这段程序更像下面的形式。

- 一个函数getLinks，可以用维基百科词条/wiki/< 词条名称> 形式的URL 链接作为参数，然后以同样的形式返回一个列表，里面包含所有的词条URL 链接。


- 一个主函数，以某个起始词条为参数调用getLinks，再从返回的URL 列表里随机选择一个词条链接，再调用getLinks，直到我们主动停止，或者在新的页面上没有词条链接了，程序才停止运行。

完整的代码如下所示：

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import re

random.seed(datetime.datetime.now())
def getLinks(articleUrl):
    html = urlopen('http://en.wikipedia.org{}'.format(articleUrl))
    bs = BeautifulSoup(html, 'html.parser')
    return bs.find('div', {'id':'bodyContent'}).find_all('a', href=re.compile('^(/wiki/)((?!:).)*$'))

links = getLinks('/wiki/Kevin_Bacon')
while len(links) > 0:
    newArticle = links[random.randint(0, len(links)-1)].attrs['href']
    print(newArticle)
    links = getLinks(newArticle)

/wiki/Internet_meme
/wiki/Owned
/wiki/Sexting
/wiki/Communications_Decency_Act
/wiki/United_States_House_of_Representatives
/wiki/Official_Congressional_Directory
/wiki/1972_in_the_United_States
/wiki/Dow_Jones_Industrial_Average
/wiki/Spanish%E2%80%93American_War
/wiki/Battle_of_Fort_Rivi%C3%A8re
/wiki/International_Standard_Book_Number
/wiki/ISO_217
/wiki/ANSI/ASME_Y14.1
/wiki/Computer-aided_design
/wiki/UNIX
/wiki/Dennis_Ritchie
/wiki/Ole-Johan_Dahl
/wiki/Nancy_Leveson
/wiki/Enterprise_Unified_Process
/wiki/Spiral_model
/wiki/Scaled_Agile_Framework
/wiki/Application_release_automation
/wiki/Software_prototyping
/wiki/Ada_(programming_language)
/wiki/Pascal/MT%2B
/wiki/Component_Library_for_Cross_Platform
/wiki/Object-oriented
/wiki/Modula-2
/wiki/International_Standard_Book_Number
/wiki/ISO_9529
/wiki/Common_Object_Request_Broker_Architecture
/wiki/List_of_ISO_romanizations
/wiki/Informal_romanizations_of_Cyrillic
/wiki/Mobile_phone
/wiki/Erna_Schneider_Hoover
/wiki/ARPANET
/wiki/Co

KeyboardInterrupt: 

导入需要的Python 库之后，程序首先做的是用系统当前时间生成一个随机数生成器。这样可以保证在每次程序运行的时候，维基百科词条的选择都是一个全新的随机路径。

然后，我们定义getLinks 函数，其参数是维基百科词条页面中/wiki/< 词条名称> 形式的URL 链接，前面加上维基百科的[域名](http://en.wikipedia.org)，再用域名中的网页获得一个BeautifulSoup 对象。之后用前面介绍过的参数抽取一列词条链接所在的标签a 并返回它们。

程序的主函数首先把[起始页面](https://en.wikipedia.org/wiki/Kevin_Bacon)里的词条链接列表（links 变量）设置成链接列表。然后用一个循环，从页面中随机找一个词条链接标签并抽取href 属性，打印这个页面链接，再把这个链接传入getLinks 函数，重新获取新的链接列表。

当然，*这里只是简单地构建一个从一个页面到另一个页面的爬虫*，**要解决“维基百科六度分隔理论”问题还有一点儿工作得做**。我们还应该存储URL 链接数据并分析数据。关于这个问题后续的解决办法，请参考第5 章内容。

#### 异常处理

虽然为了方便起见，我们在这些示例中忽略了大多数异常处理过程，但是要
注意问题随时可能发生。例如，维基百科改变了bodyContent 标签的名称怎
么办呢？（提示：那时代码就会崩溃。）

因此，这些脚本作为容易演示的示例也许可以运行得很不错，但是要真正成
为自动化产品代码，还需要增加更多的异常处理。关于异常处理的更多信
息，请参考第1 章的相关内容。

### 3.2 采集整个网站

我们实现了在一个网站上随机地从一个链接跳到另一个链接。但是，如果你需要系统地把整个网站按目录分类，或者要搜索网站上的每一个页面，怎么办？那就
得采集整个网站，那是一种非常耗费内存资源的过程，尤其是处理大型网站时，最合适的工具就是用一个数据库来储存采集的资源。但是，我们可以掌握这类工具的行为，并不需要通过大规模地运行它们。要了解更多关于数据库使用的相关知识，请参考第5 章。

那么，什么时候采集整个网站是有用的，而什么时候采集整个网站又是有害无益的呢？遍历整个网站的网络数据采集有许多好处。

- 生成网站地图

- 收集数据

为了避免一个页面被采集两次，链接去重是非常重要的。在代码运行时，把已发现的所有
链接都放到一起，并保存在方便查询的列表里（下文示例指Python 的集合set 类型）。只
有“新”链接才会被采集，之后再从页面中搜索其他链接：

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

pages = set() # 集合可以去掉重复的元素
def getLinks(pageUrl):
    global pages
    html = urlopen('http://en.wikipedia.org{}'.format(pageUrl))
    bs = BeautifulSoup(html, 'html.parser')
    for link in bs.find_all('a', href=re.compile('^(/wiki/)')):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                #We have encountered a new page
                newPage = link.attrs['href']
                print(newPage)
                pages.add(newPage)
                getLinks(newPage)
getLinks('')
            
        
    

/wiki/Wikipedia
/wiki/Wikipedia:Protection_policy#semi
/wiki/Wikipedia:Requests_for_page_protection
/wiki/Wikipedia:Requests_for_permissions
/wiki/Wikipedia:Requesting_copyright_permission
/wiki/Wikipedia:User_access_levels
/wiki/Wikipedia:Requests_for_adminship
/wiki/Wikipedia:Protection_policy#extended
/wiki/Wikipedia:Lists_of_protected_pages
/wiki/Wikipedia:Protection_policy
/wiki/Wikipedia:Perennial_proposals
/wiki/Wikipedia:Project_namespace#How-to_and_information_pages
/wiki/Wikipedia:Protection_policy#move
/wiki/Wikipedia:WPPP
/wiki/File:People_icon.svg
/wiki/Special:WhatLinksHere/File:People_icon.svg
/wiki/Help:What_links_here
/wiki/Wikipedia:Policies_and_guidelines
/wiki/Wikipedia:Shortcut
/wiki/Wikipedia:Keyboard_shortcuts
/wiki/Wikipedia:WikiProject_Kansas
/wiki/Wikipedia:WikiProject
/wiki/Wikipedia:Wikimedia_sister_projects
/wiki/Help:Interwikimedia_links
/wiki/Help:Interlanguage_links
/wiki/List_of_ISO_639-1_codes
/wiki/File:Question_book-new.svg
/wiki/Wikipedia:Protection

KeyboardInterrupt: 

为了全面地展示这个网络数据采集示例是如何工作的，我降低了在前面例子里使用的“只
寻找内链”的标准。不再限制爬虫采集的页面范围，只要遇到页面就查找所有以/wiki/ 开
头的链接，也不考虑链接是不是包含分号。（提示：词条链接不包含分号，而文档上传页
面、讨论页面之类的页面URL 链接都包含分号。）

一开始，用getLinks 处理一个空URL，其实是维基百科的主页，因为在函数里空URL 就
是http://en.wikipedia.org 。然后，遍历首页上每个链接，并检查是否已经在全局变量
集合pages 里面了（已经采集的页面集合）。如果不在，就打印到屏幕上，并把链接加入
pages 集合，再用getLinks 递归地处理这个链接。

### 收集整个网站数据

为了有效地使
用它们，在用爬虫的时候我们需要在页面上做些事情。让我们看看如何创建一个爬虫来收
集页面标题、正文的第一个段落，以及编辑页面的链接（如果有的话）这些信息。

和往常一样，决定如何做好这些事情的第一步就是先观察网站上的一些页面，然后拟定一
个采集模式。通过观察几个维基百科页面，包括词条和非词条页面，比如隐私策略之类的
页面，就会得出下面的规则。

- 所有的标题（所有页面上，不论是词条页面、编辑历史页面还是其他页面）都是在h1 → span 标签里，而且页面上只有一个h1 标签。


- 前面提到过，所有的正文文字都在div#bodyContent 标签里。但是，如果我们想更进一步获取第一段文字，可能用div#mw-content-text → p 更好（只选择第一段的标签）。这个规则对所有页面都适用，除了文件页面（例如，https://en.wikipedia.org/wiki/File:Orbit_of_274301_Wikipedia.svg ），页面不包含内容文字（content text）的部分内容。


- 编辑链接只出现在词条页面上。如果有编辑链接，都位于li#ca-edit 标签的li#caedit→ span → a 里面。

调整前面的代码，我们就可以建立一个爬虫和数据收集（至少是数据打印）的组合程序：

In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

pages = set()
def getLinks(pageUrl):
    global pages
    html = urlopen('http://en.wikipedia.org{}'.format(pageUrl))
    bs = BeautifulSoup(html, 'html.parser')
    try:
        print(bs.h1.get_text())
        print(bs.find(id ='mw-content-text').find_all('p')[0])
        print(bs.find(id='ca-edit').find('span').find('a').attrs['href'])
    except AttributeError:
        print('This page is missing something! Continuing.')
    
    for link in bs.find_all('a', href=re.compile('^(/wiki/)')):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                #We have encountered a new page
                newPage = link.attrs['href']
                print('-'*20)
                print(newPage)
                pages.add(newPage)
                getLinks(newPage)
getLinks('') 

Main Page
<p><b><a href="/wiki/Lady_Gaga" title="Lady Gaga">Lady Gaga</a></b> (born March 28, 1986) is an American singer, songwriter, and actress. She rose to prominence in 2008 with her debut album <i><a href="/wiki/The_Fame" title="The Fame">The Fame</a></i> and its singles "<a href="/wiki/Just_Dance_(song)" title="Just Dance (song)">Just Dance</a>" and "<a href="/wiki/Poker_Face_(Lady_Gaga_song)" title="Poker Face (Lady Gaga song)">Poker Face</a>". Her EP <i><a href="/wiki/The_Fame_Monster" title="The Fame Monster">The Fame Monster</a></i> followed the next year, and featured the successful singles "<a href="/wiki/Bad_Romance" title="Bad Romance">Bad Romance</a>", "<a href="/wiki/Telephone_(song)" title="Telephone (song)">Telephone</a>", and "<a href="/wiki/Alejandro_(song)" title="Alejandro (song)">Alejandro</a>". The <a href="/wiki/Born_This_Way_(song)" title="Born This Way (song)">title track</a> from her second album <i><a href="/wiki/Born_This_Way_(album)" title="Born This Way

Wikipedia:Requests for permissions
<p><span class="sysop-show" id="coordinates"><a href="/wiki/Wikipedia:Requests_for_permissions/Administrator_instructions" title="Wikipedia:Requests for permissions/Administrator instructions">Administrator instructions</a></span></p>
This page is missing something! Continuing.
--------------------
/wiki/Wikipedia:Requesting_copyright_permission
Wikipedia:Requesting copyright permission
<p>To use copyrighted material on Wikipedia, it is <i>not enough</i> that we have permission to use it on Wikipedia alone. That's because Wikipedia itself states all its material may be used by anyone, for any purpose. So we have to be sure all material is in fact licensed for that purpose, whoever provided it.</p>
This page is missing something! Continuing.
--------------------
/wiki/Wikipedia:User_access_levels
Wikipedia:User access levels
<p>The <b>user access level</b> of an editor affects their ability to perform certain actions on Wikipedia; it depends on which <

KeyboardInterrupt: 

这个for 循环和原来的采集程序基本上是一样的（除了打印一条虚线来分离不同的页面内
容之外）。

因为我们不可能确保每一页上都有所有类型的数据，所以每个打印语句都是按照数据在页
面上出现的可能性从高到低排列的。也就是说，``<h1> ``标题标签会出现在每一页上（只要能
识别，无论哪一页都有），所以我们首先试着获取它的数据。正文内容会出现在大多数页
面上（除了文件页面），因此是第二个获取的数据。“编辑”按钮只出现在标题和正文内容
都已经获取的页面上，但不是所有这类页面上都有，所以我们最后打印这类数据。

#### 不同模式应对不同需求
```
在一个异常处理语句中包裹多行语句显然是有点儿危险的。首先，你没法儿识别出究竟是哪行代码出现了异常。其次，如果有个页面没有前面的标题内
容，却有“编辑”按钮，那么由于前面已经发生异常，后面的“编辑”按钮链接就不会出现。但是，这种按照网站上信息出现的可能性高低进行排序的
方法对许多网站都是可行的，偶而会丢失一点儿数据，只要保存详细的日志就不是什么问题了。
```

你可能还发现在到目前为止所有的例子中，我们都没有“收集”那些“打印”出来的数
据。显然，命令行里显示的数据是很难进一步处理的。我们将在第5 章继续介绍信息储存
和数据库创建的内容。

### 3.3　通过互联网采集

每次在我做网络数据采集的演讲时，总有人故意问我：“你怎么建一个谷歌网站？”我的
回答通常会包含两点：“首先，你得有几十亿美元能够买得起世界上最大的数据仓库，并
把它们隐秘地放在世界各地。其次，你得写一个网络爬虫。”

谷歌在1994 年成立的时候，就是两个斯坦福大学的毕业生用一个陈旧的服务器和一个
Python 网络爬虫。

说句实在话，网络爬虫位于许多新式的网络技术领域彼此交叉的中心地带，而且你使用它
们也不需要一个大型数据仓库。要实现任何跨站的数据分析，你只要构建出可以从互联网
上无数的网页里解析和储存数据的爬虫就可以了。

就像之前的例子一样，我们后面要建立的网络爬虫也是顺着链接从一个页面跳到另一个页
面，描绘出一张网络地图。但是这一次，它们不再忽略外链，而是跟着外链跳转。我们想
看看爬虫是不是可以记录我们浏览过的每一个页面上的信息，这将是一个新的挑战。相比
我们之前做的单个域名采集，互联网采集要难得多——不同网站的布局迥然不同。这就意
味着我们必须在要寻找的信息以及查找方式上都极具灵活性。

#### 在你写爬虫随意跟随外链跳转之前，请问自己几个问题。

- 我要收集哪些数据？这些数据可以通过采集几个已经确定的网站（永远是最简单的做法）完成吗？或者我的爬虫需要发现那些我可能不知道的网站吗？


- 当我的爬虫到了某个网站，它是立即顺着下一个出站链接跳到一个新网站，还是在网站上呆一会儿，深入采集网站的内容？


- 有没有我不想采集的一类网站？我对非英文网站的内容感兴趣吗？


- 如果我的网络爬虫引起了某个网站网管的怀疑，我如何避免法律责任？（关于这个问题的更多信息请参考附录C。）

几个灵活的Python 函数组合起来就可以实现不同类型的网络爬虫，用不超过50 行代码就
可轻松地写出来：

In [12]:
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import datetime
import random

pages = set()
random.seed(datetime.datetime.now())

#Retrieves a list of all Internal links found on a page
# 获取页面所有内链的列表
def getInternalLinks(bs, includeUrl):
    includeUrl = '{}://{}'.format(urlparse(includeUrl).scheme, urlparse(includeUrl).netloc)
    internalLinks = []
    #Finds all links that begin with a "/"
    # 找出所有以"/"开头的链接
    for link in bs.find_all('a', href=re.compile('^(/|.*'+includeUrl+')')):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in internalLinks:
                if(link.attrs['href'].startswith('/')):
                    internalLinks.append(includeUrl+link.attrs['href'])
                else:
                    internalLinks.append(link.attrs['href'])
    return internalLinks
            
#Retrieves a list of all external links found on a page
# 获取页面所有外链的列表
def getExternalLinks(bs, excludeUrl):
    externalLinks = []
    #Finds all links that start with "http" that do
    #not contain the current URL
    # 找出所有以"http"或"www"开头且不包含当前URL的链接
    for link in bs.find_all('a', href=re.compile('^(http|www)((?!'+excludeUrl+').)*$')):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in externalLinks:
                externalLinks.append(link.attrs['href'])
    return externalLinks

def getRandomExternalLink(startingPage):
    html = urlopen(startingPage)
    bs = BeautifulSoup(html, 'html.parser')
    externalLinks = getExternalLinks(bs, urlparse(startingPage).netloc)
    if len(externalLinks) == 0:
        print('No external links, looking around the site for one')
        domain = '{}://{}'.format(urlparse(startingPage).scheme, urlparse(startingPage).netloc)
        internalLinks = getInternalLinks(bs, domain)
        return getRandomExternalLink(internalLinks[random.randint(0,
                                    len(internalLinks)-1)])
    else:
        return externalLinks[random.randint(0, len(externalLinks)-1)]
    
def followExternalOnly(startingSite):
    externalLink = getRandomExternalLink(startingSite)
    print('Random external link is: {}'.format(externalLink))
    followExternalOnly(externalLink)
            
followExternalOnly('http://www.jd.com')

No external links, looking around the site for one
Random external link is: http://www.hd315.gov.cn/beian/view.asp?bianhao=010202007080200026
Random external link is: http://www.baic.gov.cn
Random external link is: http://finance.qianlong.com/zt/calendar12315/index.htm
No external links, looking around the site for one
No external links, looking around the site for one


ValueError: empty range for randrange() (0,0, 0)

网站首页上并不能保证一直能发现外链。这时为了能够发现外链，就需要用一种类似前面
案例中使用的采集方法，即递归地深入一个网站直到找到一个外链才停止。

图3-1 把程序操作可视化成了一个流程图。

<img align='left' src="figures/3-1.png">

#### 不要把示例程序放进产品代码
```
我想把代码写得更完整，但是写书的时候空间和可读性非常重要，所以书中的示例程序没有包含真实产品代码中必须有的检查和异常处理。

例如，如果爬虫遇到一个网站里面一个外链都没有（虽然不太可能，但是如果程序运行的时候够长总会遇到这类情况），这时程序就会一直
在这个网站运行跳不出去，直到递归到达Python 的限制为止。

在以任何正式的目的运行代码之前，请确认你已经在可能出现问题的地方都放置了检查语句。
```

把任务分解成像“获取页面上所有外链”这样的小函数是不错的做法，以后可以方便地修
改代码以满足另一个采集任务的需求。例如，如果我们的目标是采集一个网站所有的外
链，并且记录每一个外链，我们可以增加下面的函数：

In [13]:
# Collects a list of all external URLs found on the site
allExtLinks = set()
allIntLinks = set()


def getAllExternalLinks(siteUrl):
    html = urlopen(siteUrl)
    domain = '{}://{}'.format(urlparse(siteUrl).scheme,
                              urlparse(siteUrl).netloc)
    bs = BeautifulSoup(html, 'html.parser')
    internalLinks = getInternalLinks(bs, domain)
    externalLinks = getExternalLinks(bs, domain)

    for link in externalLinks:
        if link not in allExtLinks:
            allExtLinks.add(link)
            print(link)
    for link in internalLinks:
        if link not in allIntLinks:
            allIntLinks.add(link)
            getAllExternalLinks(link)


allIntLinks.add('http://oreilly.com')
getAllExternalLinks('http://oreilly.com')

https://www.oreilly.com
http://www.oreilly.com/ideas
https://www.safaribooksonline.com/?utm_medium=content&utm_source=oreilly.com&utm_campaign=lgen&utm_content=20170601+nav
http://www.oreilly.com/conferences/
http://shop.oreilly.com/
http://members.oreilly.com
https://www.oreilly.com/topics
https://www.safaribooksonline.com/?utm_medium=content&utm_source=oreilly.com&utm_campaign=lgen&utm_content=20170505+homepage+get+started+now
https://www.safaribooksonline.com/accounts/login/?utm_medium=content&utm_source=oreilly.com&utm_campaign=lgen&utm_content=20170203+homepage+sign+in
https://www.safaribooksonline.com/live-training/?utm_medium=content&utm_source=oreilly.com&utm_campaign=lgen&utm_content=20170201+homepage+take+a+live+online+course
https://www.safaribooksonline.com/learning-paths/?utm_medium=content&utm_source=oreilly.com&utm_campaign=lgen&utm_content=20170201+homepage+follow+a+path
https://www.safaribooksonline.com/?utm_medium=content&utm_source=oreilly.com&utm_campaign=lgen&utm_c

KeyboardInterrupt: 

这段代码可以看出两个循环——一个是收集内链，一个是收集外链——然后彼此连接起来
工作，程序的流程如图3-2 所示。

<img align='left' src="figures/3-2.png">

### ``写代码之前拟个大纲或画个流程图是很好的编程习惯，这么做不仅可以为你后期处理节省很多时间，更重要的是可以防止自己在爬虫变得越来越复杂时乱了分寸。``

### 3.4　用Scrapy采集

写网络爬虫的挑战之一是你经常需要不断地重复一些简单任务：找出页面上的所有链接，
区分内链与外链，跳转到新的页面。掌握这些基本模式非常有用，从零开始编写也完全可
行，不过有几个工具可以帮你自动处理这些细节。

Scrapy 就是一个帮你大幅度降低网页链接查找和识别工作复杂度的Python 库，它可以
让你轻松地采集一个或多个域名的信息。